In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from datetime import datetime
from sklearn.metrics import jaccard_score,accuracy_score,f1_score
import tensorflow as tf
import os
from PIL import Image as im
from google.colab import drive
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:


def read_input(file_):
    img = cv2.imread(folder + 'train_partition_images/full_color_training_' + str(file_) + '.png') #full colour rgb image ( 3 dimensional )
    #print(img)
    img = np.reshape(img, (256*256,3)) 
    #print("img_dim: ",img.ndim) 
    return img

def read_test_input(file_):
    img = cv2.imread(folder + 'test_partition_images/full_color_training_' + str(file_) + '.png') #full colour rgb image ( 3 dimensional )
    #print(img)
    img = np.reshape(img, (256*256,3)) 
    #print("img_dim: ",img.ndim) 
    return img

In [ ]:
def read_labels(file_):
    label_img = cv2.imread(folder + 'train_partition_ground_truth/ground_truth_training_' + str(file_) + '.png')
    #print("label_img_before: ",label_img.ndim)
    label = label_img[:,:,0] #one channel
    labels = np.reshape(label, (256*256))
    #print(label)
    #print("label_dim: ",label.ndim)
    return labels

def read_test_labels(file_):
    label_img = cv2.imread(folder + 'test_partition_ground_truth/ground_truth_training_' + str(file_) + '.png')
    #print("label_img_before: ",label_img.ndim)
    label = label_img[:,:,0] #one channel
    labels = np.reshape(label, (256*256))
    #print(label)
    #print("label_dim: ",label.ndim)
    return labels

In [ ]:
def to_string(files,i):
    return files['name'][i]


In [ ]:
def train_scaling(X_train):
  
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  
  pca.fit(X_train)

  X_train = pca.transform(X_train)
  X_train = np.array(X_train)
  return X_train

In [ ]:
def test_scaling(X_test):
  
  X_test = scaler.transform(X_test)
  X_test = pca.transform(X_test)
  X_test= np.array(X_test)
  return X_test

In [ ]:
start_time = datetime.now()
print(str(start_time))


2021-07-21 20:23:45.751826


In [ ]:
folder='/content/drive/MyDrive/SequoiaMulti_30/'


In [ ]:
files =pd.read_csv(folder + 'train.csv')


In [ ]:
#image=cv2.imread('/content/drive/MyDrive/SequoiaMulti_30/train_partition_images/full_color_training_1.png')
#print(image)
#print(image.shape)
#print(image[:256,:256,0])
#cv2_imshow(image[:256,:256,])

In [ ]:
num = to_string(files,0)
X_train = read_input(num)
y_train = read_labels(num)

In [ ]:
scaler = StandardScaler()
pca = PCA(0.95)

In [ ]:
model = SGDClassifier(max_iter=1000,loss='hinge',fit_intercept=False)

In [ ]:
for file_ in range(1,len(files['name'])):
    num = to_string(files,file_)
    if(file_ == 1001 or file_ == 2001):
        X_train = read_input(num)
        y_train = read_labels(num)
        continue
        
    if(file_%100 == 0):
        print(num)
        #print(file_)
    #print(num)
    X_train = np.concatenate((X_train,read_input(num)))
    y_train = np.concatenate((y_train,read_labels(num)))
    #break
    if( file_%1000==0 and file_!=0):
        print('Done Loading:')
        #model_log = model.partial_fit(X_train,y_train,classes=np.unique(y_train))
        X_train=train_scaling(X_train)
        
        model_log = model.fit(X_train,y_train)
        #break




101
201
301
401
501
601
701
801
901
1001
Done Loading:
1101
1201
1301
1401
1501
1601
1701
1801


In [ ]:

print(X_train.shape)
print(y_train.shape)

pickle.dump(model_log, open(folder + 'linear_svm_' + str(file_) + '_images_fold_' + '.sav', 'wb'))




(57278464, 3)
(57278464,)


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
model = pickle.load(open(folder + 'linear_svm_1874_images_fold_.sav', 'rb'))


In [ ]:
test_files = pd.read_csv(folder + 'test.csv') 
print(len(test_files['name']))

450


In [ ]:
predictions_folder = folder + 'Predictions_' + '/'
if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder)

In [ ]:
height=256
width=256
final_img = 0
points = height*width

In [ ]:
num = to_string(test_files,0)
X_test=read_test_input(num)
X_test=test_scaling(X_test)
y_pred_array = model.predict(X_test)
y_pred_img = np.reshape(y_pred_array,(height,width,1))
final_img = np.stack((y_pred_img,y_pred_img,y_pred_img),axis=2)
final_img = final_img[:,:,:,0]
y_color = np.where(final_img==1,60,final_img)
y_color = np.where(y_color==2,255,y_color)
cv2.imwrite(predictions_folder + 'pred_' + str(num) + '.png', y_color)
#print(np.unique(y_color))

True

In [ ]:
for file_ in range(1,len(test_files['name'])):
    num = to_string(test_files,file_)
    
        
    #if(file_%100 == 0):
        #print(file_)
    X_test=read_test_input(num)
    X_test=test_scaling(X_test)
    y_pred = model.predict(X_test)
    
    
    y_pred_array = np.concatenate((y_pred_array,y_pred))
    #Converting to IMAGE
    y_pred_img = np.reshape(y_pred,(height,width,1))
    #print(y_pred_img.shape)
    
    final_img = np.stack((y_pred_img,y_pred_img,y_pred_img),axis=2)
    #print(final_img.shape)
    final_img = final_img[:,:,:,0]
    #print(final_img.shape)
    #print(np.unique(final_img))
    y_color = np.where(final_img==1,60,final_img)
    y_color = np.where(y_color==2,255,y_color)
    cv2.imwrite(predictions_folder + 'pred_' + str(num) + '.png', y_color)
    #print(np.unique(y_color))
    #plt.imshow(y_color)
    #plt.show()
    #if(file_ == 2):
    #  break
        


In [ ]:
print(len(test_files['name']))

450


In [ ]:
num = to_string(test_files,0)
y_test_array = read_test_labels(num)
print(y_test_array)


[0 0 0 ... 0 0 0]


In [ ]:
#TEST ARRAY FOR GROUND TRUTH LABELS : Y_TRUE

for file_ in range(1,len(test_files['name'])):
    num = to_string(test_files,file_)
    if(file_ == 1001 or file_ == 2001):
        
        y_test_array = read_test_labels(num)
        continue
        
    if(file_%100 == 0):
        print(num)
        #print(file_)
    #print(num)
   
    y_test_array = np.concatenate((y_test_array,read_test_labels(num)))
    #break
    if( file_%1000==0 and file_!=0):
        print('Done Loading:')




101
201
301
401


In [ ]:
print(type(y_pred_array))
print(y_pred_array.ndim)
print(type(y_pred_array))
print(y_pred_array[0:10])
print(y_pred_array[1:10])
print(y_pred_array.shape)
np.unique(y_pred_array)

<class 'numpy.ndarray'>
1
<class 'numpy.ndarray'>
[1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0]
(29491200,)


array([0, 1, 2], dtype=uint8)

In [ ]:
print(type(y_test_array))
print(y_test_array.ndim)
print(type(y_test_array))
print(y_test_array[:10])
print(y_test_array.shape)
np.unique(y_test_array)

<class 'numpy.ndarray'>
1
<class 'numpy.ndarray'>
[0 0 0 0 0 0 0 0 0 0]
(29491200,)


array([0, 1, 2], dtype=uint8)

In [ ]:
comp=y_pred_array==y_test_array
print(comp.all())

False


In [ ]:
jaccard_scores = [0,0]
f1_scores = [0,0]

In [ ]:
def the_jaccard_metric(y_pred_array,y_test_array): #double check metric functions
  # do per image, add metric per image, find avg
  gap_begin=0
  gap=65536
  for i in range(1,450):
    jaccard_scores[0] += jaccard_score(y_test_array[gap_begin:gap], y_pred_array[gap_begin:gap],average='macro')
    jaccard_scores[1] += jaccard_score(y_test_array[gap_begin:gap], y_pred_array[gap_begin:gap],average='weighted')
    gap_begin=gap+1
    gap+=65536
    print(gap)
  return jaccard_scores
    

In [ ]:
def the_f1_metric(y_pred_array,y_test_array): #double check metric functions
  gap_begin=0
  gap=65536 
  for i in range(1,450):
    f1_scores[0] += f1_score(y_test_array[gap_begin:gap], y_pred_array[gap_begin:gap],average='macro')
    f1_scores[1] += f1_score(y_test_array[gap_begin:gap], y_pred_array[gap_begin:gap],average='weighted')
    gap_begin=gap+1
    gap+=65536 
  return f1_scores

In [ ]:
a = np.array([[1, 2]])
b = np.array([[5, 6]])
c=np.concatenate((a, b), axis=0)
c[0]

array([1, 2])

In [ ]:
my_jaccard_metric=the_jaccard_metric(y_pred_array,y_test_array)

131072
196608
262144
327680
393216
458752
524288
589824
655360
720896
786432
851968
917504
983040
1048576
1114112
1179648
1245184
1310720
1376256
1441792
1507328
1572864
1638400
1703936
1769472
1835008
1900544
1966080
2031616
2097152
2162688
2228224
2293760
2359296
2424832
2490368
2555904
2621440
2686976
2752512
2818048
2883584
2949120
3014656
3080192
3145728
3211264
3276800
3342336
3407872
3473408
3538944
3604480
3670016
3735552
3801088
3866624
3932160
3997696
4063232
4128768
4194304
4259840
4325376
4390912
4456448
4521984
4587520
4653056
4718592
4784128
4849664
4915200
4980736
5046272
5111808
5177344
5242880
5308416
5373952
5439488
5505024
5570560
5636096
5701632
5767168
5832704
5898240
5963776
6029312
6094848
6160384
6225920
6291456
6356992
6422528
6488064
6553600
6619136
6684672
6750208
6815744
6881280
6946816
7012352
7077888
7143424
7208960
7274496
7340032
7405568
7471104
7536640
7602176
7667712
7733248
7798784
7864320
7929856
7995392
8060928
8126464
8192000
8257536
8323072
838860

In [ ]:
my_f1_metric=the_f1_metric(y_pred_array,y_test_array)

In [ ]:
print("F1 Metric, macro: ", my_f1_metric[0]/450)
print("Jaccard Metric, macro: ",my_jaccard_metric[0]/450 )
print("F1 Metric, weighted: ",my_f1_metric[1]/450 )
print("Jaccard Metric, weighted: ",my_jaccard_metric[1]/450 )

F1 Metric, macro:  0.41936291922546726
Jaccard Metric, macro:  0.3665220112095642
F1 Metric, weighted:  0.6559805565720129
Jaccard Metric, weighted:  0.6059308706552987


In [ ]:
end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))
print (str(end_time))


Duration: 0:54:53.317959
2021-07-21 21:18:39.069785
